In [ ]:
const DATA_FOLDER = './data/axe';
const OUTPUT_FOLDER = './notebooks/dashboard'

In [ ]:
const fs = require('fs');
const data = JSON.parse(fs.readFileSync(`${DATA_FOLDER}/keyboard-accessibility-results.json`, 'utf8'));

// console.log(data)

In [3]:
let urlViolationCounts = {};
data.forEach(record => {
    let url = record.url;
    let violationCount = 0;

    record.axeResults.violations.forEach(violation => {
        // if (!violation.tags.includes("cat.keyboard")) {
        //     continue;
        // }

        // violationCount++; // this is just for the type, not the specific count for each type

        violationCount = violationCount + +violation.nodes.length;

    });

    urlViolationCounts[url] = violationCount;
});
// console.log(urlViolationCounts)

// Sort the urlViolationCounts from greatest to least
const sortedEntries = Object.entries(urlViolationCounts).sort((a, b) => b[1] - a[1]);

const labels = sortedEntries.map(entry => entry[0]); // Extract sorted URLs
const dataPoints = sortedEntries.map(entry => entry[1]); // Extract sorted violation counts


// const labels = Object.keys(urlViolationCounts);
// const dataPoints = Object.values(urlViolationCounts);

const htmlContent = `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Violations Chart</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        /* Increase the canvas height for more space for each bar */
        #myChart {
            height: 3000px; /* Adjust as needed for your data */
        }
    </style>
</head>
<body>
    <canvas id="myChart" width="400" height="200"></canvas>
    <script>
        const ctx = document.getElementById('myChart').getContext('2d');
        new Chart(ctx, {
            type: 'bar',
            data: {
                labels: ${JSON.stringify(labels)},
                datasets: [{
                    // label: 'Keyboard Violations Per URL',
                    label: 'Violations Per URL',
                    data: ${JSON.stringify(dataPoints)},
                    backgroundColor: 'rgba(54, 162, 235, 0.2)',
                    borderColor: 'rgba(54, 162, 235, 1)',
                    borderWidth: 1,
                    barThickness: 'flex',
                }]
            },
            options: {
                indexAxis: 'y', // This changes the chart to a horizontal bar chart
                scales: {
                    y: {
                        beginAtZero: true,
                        // display:false
                    },
                    x: {
                        display: false // This hides the x-axis labels
                    }
                },
                responsive: true,
                maintainAspectRatio: false,
            }
        });
    </script>
</body>
</html>`;

// console.log(htmlContent)

fs.writeFileSync('outputs/violations_chart.html', htmlContent, 'utf8');
// console.log('HTML file has been generated');

In [4]:
// Accumulate violation counts per URL
let violationCounts = {};

data.forEach(record => {
    // let count = record.axeResults.violations.length;
    let count = 0;
    record.axeResults.violations.forEach(violation => {
        count += violation.nodes && violation.nodes.length || 0;
    })
    violationCounts[count] = (violationCounts[count] || 0) + 1;
});

In [ ]:
// Extract counts and the number of URLs for those counts to create histogram data
let violationLabels = Object.keys(violationCounts).map(Number).sort((a, b) => a - b);
let urlCounts = violationLabels.map(label => violationCounts[label]);

// Prepare the HTML content for the histogram
let violationsHtmlContent = `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Violation Histogram</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>
    <canvas id="violationHistogram" width="400" height="200"></canvas>
    <script>
        const ctx = document.getElementById('violationHistogram').getContext('2d');
        new Chart(ctx, {
            type: 'bar',
            data: {
                labels: ${JSON.stringify(violationLabels)},
                datasets: [{
                    label: 'Violations Histogram',
                    data: ${JSON.stringify(urlCounts)},
                    backgroundColor: 'rgba(75, 192, 192, 0.2)',
                    borderColor: 'rgba(75, 192, 192, 1)',
                    borderWidth: 1
                }]
            },
            options: {
                scales: {
                    x: {
                        title: {
                            display: true,
                            text: 'Number of Violations'
                        }
                    },
                    y: {
                        title: {
                            display: true,
                            text: 'Number of URLs'
                        },
                        beginAtZero: true
                    }
                }
            }
        });
    </script>
</body>
</html>
`;

// Write the HTML content to an HTML file
fs.writeFileSync(`${OUTPUT_FOLDER}/violation_histogram.html`, violationsHtmlContent, 'utf8');
// console.log('Histogram HTML file has been generated');

In [ ]:
// Assume data is properly parsed and available
let violationTypeCounts = {};

// Count each type of violation
data.forEach(record => {
    record.axeResults.violations.forEach(violation => {
        const violationType = violation.id;
        violationTypeCounts[violationType] = (violationTypeCounts[violationType] || 0);
        
        // now do the addition
        const violationOccurrences = violation.nodes && violation.nodes.length || 0;
        violationTypeCounts[violationType] = violationTypeCounts[violationType] + violationOccurrences;
    });
});

// Convert the counts object to an array of [type, count] pairs
const violationTypeArray = Object.entries(violationTypeCounts);

// Sort the array by counts in descending order
violationTypeArray.sort((a, b) => b[1] - a[1]);

// Extract sorted types and counts into separate arrays
const violationTypesSorted = violationTypeArray.map(item => item[0]);
const typeCountsSorted = violationTypeArray.map(item => item[1]);

const barChartHtml = `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Violation Types Bar Chart</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        /* Increase the canvas height for more space for each bar */
        #violationTypesChart {
            height: 1000px; /* Adjust as needed for your data */
        }
    </style>
</head>
<body>
    <canvas id="violationTypesChart" width="400" height="200"></canvas>
    <script>
        const ctx = document.getElementById('violationTypesChart').getContext('2d');
        new Chart(ctx, {
            type: 'bar',
            data: {
                labels: ${JSON.stringify(violationTypesSorted)},
                datasets: [{
                    label: 'Number of Violations',
                    data: ${JSON.stringify(typeCountsSorted)},
                    backgroundColor: 'rgba(153, 102, 255, 0.2)',
                    borderColor: 'rgba(153, 102, 255, 1)',
                    borderWidth: 1,
                    barThickness: 'flex',
                }]
            },
            options: {
                indexAxis: 'y', // This changes the chart to a horizontal bar chart
                scales: {
                    x: {
                        title: {
                            display: true,
                            text: 'Violation Types'
                        }
                    },
                    y: {
                        title: {
                            display: true,
                            text: 'Number of Violations'
                        },
                        ticks: {
                            autoSkip: false, // Ensure all labels are shown
                        },
                        beginAtZero: true
                    }
                },
                responsive: true,
                maintainAspectRatio: false,
                plugins: {
                    legend: {
                        position: 'top',
                    }
                }
            }
        });
    </script>
</body>
</html>
`;

// Generate the HTML file
fs.writeFileSync(`${OUTPUT_FOLDER}/violation_types_chart.html`, barChartHtml, 'utf8');
// console.log('Bar chart for violation types has been generated');

In [ ]:
// Doughnut chart for impact levels
// Aggregate Impact Levels
let impactLevelCounts = {};

data.forEach(record => {
    record.axeResults.violations.forEach(violation => {
        const impactLevel = violation.impact;
        impactLevelCounts[impactLevel] = (impactLevelCounts[impactLevel] || 0) + 1;
    });
});

// Extract impact levels and their counts
const impactLabels = Object.keys(impactLevelCounts);
const impactCounts = Object.values(impactLevelCounts);

// Prepare the HTML content for the Doughnut Chart
const doughnutChartHtml = `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Impact Levels Doughnut Chart</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>
    <canvas id="impactLevelsChart" width="400" height="400"></canvas>
    <script>
        const ctx = document.getElementById('impactLevelsChart').getContext('2d');
        new Chart(ctx, {
            type: 'doughnut',
            data: {
                labels: ${JSON.stringify(impactLabels)},
                datasets: [{
                    label: 'Impact Levels',
                    data: ${JSON.stringify(impactCounts)},
                    backgroundColor: [
                        'rgba(255, 99, 132, 0.2)',
                        'rgba(54, 162, 235, 0.2)',
                        'rgba(255, 206, 86, 0.2)',
                        'rgba(75, 192, 192, 0.2)'
                    ],
                    borderColor: [
                        'rgba(255, 99, 132, 1)',
                        'rgba(54, 162, 235, 1)',
                        'rgba(255, 206, 86, 1)',
                        'rgba(75, 192, 192, 1)'
                    ],
                    borderWidth: 1
                }]
            },
            options: {
                responsive: true,
                plugins: {
                    legend: {
                        position: 'top',
                    },
                    title: {
                        display: true,
                        text: 'Distribution of Impact Levels'
                    }
                }
            }
        });
    </script>
</body>
</html>
`;

// Write the HTML content to a file
fs.writeFileSync(`${OUTPUT_FOLDER}/impact_levels_doughnut_chart.html`, doughnutChartHtml, 'utf8');
// console.log('Doughnut chart for impact levels has been generated');

In [ ]:
// Prepare the data matrix and count total occurrences for sorting
let violationCountMatrix = {};
let violationTypeCountsForMatrix = {};

// Count violations per URL and total counts per type
data.forEach(record => {
    if (!violationCountMatrix[record.url]) {
        violationCountMatrix[record.url] = {};
    }
    
    record.axeResults.violations.forEach(violation => {
        if (!violationCountMatrix[record.url][violation.id]) {
            violationCountMatrix[record.url][violation.id] = 0;
        }
        violationCountMatrix[record.url][violation.id]++;
        
        if (!violationTypeCountsForMatrix[violation.id]) {
            violationTypeCountsForMatrix[violation.id] = 0;
        }
        violationTypeCountsForMatrix[violation.id]++;
    });
});

// Sort violation types by total count in descending order
const sortedViolationTypes = Object.entries(violationTypeCountsForMatrix)
    .sort((a, b) => b[1] - a[1])
    .map(entry => entry[0]);

// Extract URLs for labeling
const urls = Object.keys(violationCountMatrix);

// Construct the heatmap data array
let heatmapData = [];

urls.forEach((url, i) => {
    sortedViolationTypes.forEach((violationType, j) => {
        const count = violationCountMatrix[url][violationType] || 0;
        heatmapData.push({ x: j, y: i, v: count });
    });
});

// Prepare the HTML content for the heatmap
let heatmapHtml = `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>URLs and Violation Types Heatmap</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/chartjs-chart-matrix@1.1"></script>
</head>
<body>
    <canvas id="violationHeatmap" width="800" height="600"></canvas>
    <script>
        const sortedViolationTypes = ${JSON.stringify(sortedViolationTypes)};
        const ctx = document.getElementById('violationHeatmap').getContext('2d');
        const heatmap = new Chart(ctx, {
            type: 'matrix',
            data: {
                // Use sorted violation types
                labels: {
                    y: ${JSON.stringify(urls)},
                    x: ${JSON.stringify(sortedViolationTypes)}
                },
                datasets: [{
                    label: 'Violation Heatmap',
                    data: ${JSON.stringify(heatmapData)},
                    backgroundColor(context) {
                        const value = context.raw.v;
                        const alpha = Math.min(0.8, value / 10); // Normalize alpha; adjust divisor as needed
                        return Chart.helpers.color('blue').alpha(alpha).rgbString();
                    },
                    width: ({ chart }) =>
                        (chart.chartArea || {}).width / ${sortedViolationTypes.length},
                    height: ({ chart }) =>
                        (chart.chartArea || {}).height / ${urls.length}
                }]
            },
            options: {
                scales: {
                    x: {
                        title: {
                            display: true,
                            text: 'Violation Types'
                        },
                        // Map tick values to the index in sortedViolationTypes
                        ticks: {
                            autoSkip: false,
                            maxRotation: 90, // Set max rotation
                            minRotation: 90,  // Set min rotation to achieve a vertical label
                            callback: function(value) {
                                return sortedViolationTypes[value]; // Return the text label instead of the index
                            }
                        }
                    },
                    y: {
                        title: {
                            display: true,
                            text: 'URLs'
                        },
                        ticks: {
                            autoSkip: false
                        }
                    }
                },
                plugins: {
                    legend: {
                        display: true
                    },
                    tooltip: {
                        callbacks: {
                            label(item) {
                                const type = sortedViolationTypes[item.raw.x];
                                const url = urls[item.raw.y];
                                const count = item.raw.v;
                                return \`\${type} at \${url}: \${count} violations\`;
                            }
                        }
                    }
                },
                animation: {
                    duration: 0
                }
            }
        });
    </script>
</body>
</html>
`;

// Write the HTML content to an HTML file
fs.writeFileSync(`${OUTPUT_FOLDER}/violation_heatmap.html`, heatmapHtml, 'utf8');
// console.log('Heatmap for URLs and Violation Types has been generated');

In [ ]:
// Assume data is properly parsed and available
entryViolations = "";

// Count each type of violation
data.forEach(record => {

    let violationCount = 0;
    let violationTypesString = "";
    let violationTypeCounts = {};
    record.axeResults.violations.forEach(violation => {
        violationCount += violation.nodes && violation.nodes.length || 0; // basic violation count

        let violationType = violation.id;

        if (violation.tags.includes("cat.keyboard")) {
            violationType += " *"
        }
        
        
        violationTypeCounts[violationType] = (violationTypeCounts[violationType] || 0); // violation counts by type
        
        // now do the addition
        const violationOccurrences = violation.nodes && violation.nodes.length || 0;
        violationTypeCounts[violationType] = +violationTypeCounts[violationType] + violationOccurrences;
    })

    for (const [violationName, violationCount] of Object.entries(violationTypeCounts)) {
        violationTypesString += `
            <dt>${violationName}</dt>
            <dd>${violationCount}</dd>
        `
    };

    entryString = ``;
    entryString += `
    <dl>
        <dt>Name</dt>
        <dd><h2>${record.name}</h2></dd>
        <dt>Violations</dt>
        <dd>${violationCount}</dd>
        <dt>Violation types</dt>
        <dd>
            <dl>
                ${violationTypesString}
            </dl>
        </dd>
        
    </dl>
    `;

    entryViolations += entryString;
});

// console.log(entryViolations);

// Prepare the HTML content for the entry violations list
violationListHTML = `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Violations by Tool</title>
</head>
<body>
<h1>Violations by Tool</h1>
<p>* = keyboard violation</p>
${entryViolations}
</body>
`;

// Write the HTML content to an HTML file
fs.writeFileSync(`${OUTPUT_FOLDER}/violation_list.html`, violationListHTML, 'utf8');
// console.log('Heatmap for URLs and Violation Types has been generated'
